In [2]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_personel_columns
from lib.textract import parse_textract_datetime

In [203]:
cprr = pd.read_csv("../data/Baton Rouge Police Department/Baton Rouge_PD_CPRR_2018.csv", encoding="latin1")
cprr.rename(columns=lambda x: x.strip(), inplace=True)
cprr.rename(columns={"Comptaint": "Complaint"}, inplace=True)

# manual edits
cprr.iloc[4, 2] = "Administrative Review"
cprr.iloc[12, 5] = cprr.iloc[12, 5] + "P10467 Patrol 3rd District"
cprr.iloc[12, 6] = cprr.iloc[12, 6] + "Orders - (Pursuit) - 40"
for ind in [16, 23, 25, 29]:
    m = re.match(r"^(.+) (\d+\:\d+) *$", cprr.iloc[ind-1, 6])
    cprr.iloc[ind-1, 6] = m.group(1)
    cprr.iloc[ind, 6] = "%s %s" % (m.group(2), cprr.iloc[ind, 6])
for ind in [39]:
    m = re.match(r"^(\d+) (.+)$", cprr.iloc[ind+1, 6])
    cprr.iloc[ind, 6] = cprr.iloc[ind, 6] + m.group(1)
    cprr.iloc[ind+1, 6] = m.group(2)
cprr.iloc[24, 5] = cprr.iloc[24, 5][21:]
cprr.iloc[38, 5] = cprr.iloc[38, 5] + "Operational Services"
cprr.iloc[66, 6] = cprr.iloc[66, 6] + "Orders - (Pursuit) - 40"
cprr.iloc[72, 0] = "2018"
cprr.iloc[72, 1] = "35"
cprr.iloc[72, 2] = "Administrative Review"
cprr.iloc[72, 5] = cprr.iloc[72, 5][:-1]
cprr.iloc[72, 6] = cprr.iloc[72, 6][:-1]
cprr.iloc[83, 5] = "%s%s" % (cprr.iloc[83, 5], cprr.iloc[84, 5])
cprr.iloc[83, 6] = "%s%s" % (cprr.iloc[83, 6], cprr.iloc[84, 6])
cprr.iloc[128, 5] = "%s%s" % (cprr.iloc[128, 5], cprr.iloc[133, 4])
cprr.iloc[128, 6] = "%s%s" % (cprr.iloc[128, 6], cprr.iloc[133, 5])
cprr.iloc[115, 6] = "%s%s" % (cprr.iloc[115, 6], cprr.iloc[129, 6])
for ind in [134, 135]:
    cprr.iloc[ind, 6] = cprr.iloc[ind, 5]
    cprr.iloc[ind, 5] = cprr.iloc[ind, 4]
    cprr.iloc[ind, 4] = ""

# normalize
cprr.loc[:, "IA Year"] = cprr["IA Year"].str.replace("-", "").str.strip()
cprr.loc[:, "Status"] = cprr["Status"].str.replace("-", "").str.strip()\
    .str.title()
cprr.loc[:, "Received"] = cprr["Received"].str.replace("-", "").str.strip()
cprr.loc[:, "Occur Date"] = cprr["Occur Date"].str.replace("-", "")\
    .str.strip().fillna("")
cprr.loc[:, "Officer Name"] = cprr["Officer Name"].str.strip()\
    .str.replace(r"(\.|,) +(PC?\d+)", r", \2")\
    .str.replace(", JR", ". JR", regex=False)\
    .str.replace(" Il ", " II ", regex=False)\
    .str.replace(r"(\d+)\. ", r"\1 ")\
    .str.replace("none entered", "", regex=False).fillna("")
cprr.loc[:, "Complaint"] = cprr.loc[:, "Complaint"]\
    .str.replace(r"\< *\d+.+$", "").str.strip()\
    .str.replace(r"^(\d+)(?:\.|,)(\d+)", r"\1:\2")\
    .str.replace(r" \.(\d+)", r" \1")\
    .str.replace(r" of$", "").str.replace("Use Force", "Use of Force", regex=False)\
    .str.replace(r" ?- ", " ").str.replace(" lo ", " to ", regex=False)\
    .str.replace("Emply", "Empty", regex=False)\
    .str.replace("Oul ", "Out ", regex=False).fillna("")
cprr.loc[:, "Action"] = cprr["Action"]\
    .where(~cprr["Action"].str.isupper().fillna(False), cprr["Action"].str.title())\
    .str.replace(" lo ", " to ", regex=False)\
    .str.replace("Suspension.", "Suspension,", regex=False)\
    .str.replace("Exonerated;", "Exonerated:", regex=False)\
    .str.replace("Sgl. ", "Sgt. ", regex=False).str.strip().fillna("")
cprr.loc[:, "Disposition"] = cprr["Disposition"].str.replace(r" ?- ", " ")\
    .str.replace(r"\< *\d+.+$", "").str.replace(r"^\.", "")\
    .str.replace("Exoneraled", "Exonerated", regex=False)\
    .str.replace("Sustaned", "Sustained", regex=False).str.strip().fillna("")

cprr.drop([84, 129, 133], inplace=True)
cprr.reset_index(drop=True, inplace=True)
cprr


,IA Year,1A Seq,Status,Received,Occur Date,Officer Name,Complaint,Action,Disposition
0,2018,2,Administrative Review,1/2/18,12/30/17,"BARCELONA JESSE R, P1254 Cib Street Crimes Unit",3:17 Carrying Out Orders / General Orders (Pur...,Exonerated,Exonerated
1,2018,4,Administrative Review,1/9/18,1/9/18,"ACREE JASON K, P10280 Cib Narcotics",3:17 Carrying Out Orders / General Orders (Pur...,Not Sustained,Not Sustained
2,2018,3,Office Investigation,1/9/18,12/3/17,"RIMES MELISSA J, P10476 Op Service Comm Center",2:4 Abuse of Sick Leave 66,Resigned,Office Investigation
3,2018,3,Office Investigation,1/9/18,12/3/17,"RIMES MELISSA J, P10476 Op Service Comm Center",2:4 Abuse of Sick Leave 66,Office Investigation,Office Investigation
4,2018,5,Administrative Review,1/10/18,1/10/18,"HAMADEH YUSEFF G, P10578 Cib Cib",3:20 Use of Force / Hard Empty Hand 53,Not Sustained,Not Sustained
...,...,...,...,...,...,...,...,...,...
142,2019,3,Administrative Review,1/30/19,3/27/18,"CLARK KENNETH M, P1245 Patrol 2nd District",2:2 Command of Temper 13,Letter of Reprimand,Not Sustained
143,2019,3,Administrative Review,1/30/19,3/27/18,"CLARK KENNETH M, P1245 Patrol 2nd District",2:13 Respect of Fellow Officers 22,Letter of Reprimand,Sustained
144,2019,3,Administrative Review,1/30/19,3/27/18,"CLARK KENNETH M, P1245 Patrol 2nd District",3:17 Carrying Out Orders / General Orders (Pur...,Letter of Reprimand,Sustained
145,2019,3,Administrative Review,1/30/19,3/27/18,"CLARK KENNETH M, P1245 Patrol 2nd District",3:18 Insubordination 43,Letter of Reprimand,Not Sustained


In [209]:
cprr.info()
# cprr[cprr.iloc[:, 7].str.contains("Training;", na=False)]
# cprr[cprr.iloc[:, 5] == 'Operational Services STEELE JR. DONALD N/A, P10526 Patrol 2nd District']
# cprr.iloc[72]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IA Year       147 non-null    object
 1   1A Seq        147 non-null    object
 2   Status        147 non-null    object
 3   Received      147 non-null    object
 4   Occur Date    147 non-null    object
 5   Officer Name  147 non-null    object
 6   Complaint     147 non-null    object
 7   Action        147 non-null    object
 8   Disposition   147 non-null    object
dtypes: object(9)
memory usage: 10.5+ KB
